select from the old test.csv the "fake"images (class == 1), that contains both the generated images and the manipulated images

In [2]:

import os
import shutil
import pandas as pd

df = pd.read_csv("C:/Users/nello/Documents/vscode_projects/Thesis/4_deepfake_detection/csv/test.csv")

# Filter the DataFrame based on conditions: exclude images starting with "SD", "DL" or "GL" and have class == 1
filtered_df = df[df['class'] == 1]

selected_ids = filtered_df['id']
selected_ids.to_csv("class1.csv")

exclude the test images that were belonging to class 4 originally (manipulated images)

In [3]:
import pandas as pd

# Load the DataFrame from the original CSV file
df = pd.read_csv("class1.csv")

# Filter the DataFrame to include only rows where the id column does not start with "SD_", "DL_", or "GL_"
filtered_df = df[~df['id'].str.startswith(('SD_', 'DL_', 'GL_'))]

# Save the filtered DataFrame with only the id column to a new CSV file
filtered_df[['id']].to_csv("class4_to_exclude.csv", index=False)


copy from dataset_after_merging_WITH_DUPLICATES to new_dataset/test

In [8]:
import os
import shutil
import pandas as pd

selected_ids = pd.read_csv("C:/Users/nello/Documents/vscode_projects/Thesis/4_deepfake_detection/csv/test.csv")["id"]

# Load excluded IDs from the second CSV file
excluded_ids = pd.read_csv("class4_to_exclude.csv")["id"]

# Exclude IDs from selected_ids if they are in the excluded_ids list
selected_ids = selected_ids[~selected_ids.isin(excluded_ids)]

print(len(selected_ids))

# Get a list of all the image filenames in the folder (without extensions)
folder_path = "C:/Users/nello/Desktop/TESI/dataset_after_merging_WITH_DUPLICATES"
folder_images = set(os.path.splitext(filename)[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = set(selected_ids).intersection(folder_images)

destination_folder = 'C:/Users/nello/Desktop/TESI/dataset/test'

num_copied_images = 0
for image_name in matching_images:
    source_path = os.path.join(folder_path, image_name + ".jpg")
 
    destination_path = os.path.join(destination_folder, image_name + ".jpg")  
    
    #shutil.copy(source_path, destination_path)
    
    num_copied_images += 1

print("Total number of copied images:", num_copied_images)

51567
Total number of copied images: 51567


also save the new test csv for the new dataset, starting from old text csv and excluding the manipulated images (class 4)

In [11]:
import pandas as pd

df_a = pd.read_csv("C:/Users/nello/Documents/vscode_projects/Thesis/4_deepfake_detection/csv/test.csv")
df_b = pd.read_csv("class4_to_exclude.csv")

# Perform a left join on "id" column
merged_df = pd.merge(df_a, df_b, on="id", how="left", indicator=True)

# Filter rows where the merge resulted in null values in DataFrame B
filtered_df = merged_df[merged_df['_merge'] == 'left_only']

# Select only the columns from DataFrame A and save to CSV file C
filtered_df.drop(columns='_merge').to_csv("csv/test/test.csv", index=False)

now rename the columns

In [14]:
import pandas as pd

df = pd.read_csv("csv/test/test.csv")

# Rename the columns
df = df.rename(columns={"real_image": "pristine_image", "fake_image": "generated_image"})

df.to_csv("csv/test/test.csv", index=False)

check whether the generated images are just the only that we generate (starts with SD, GL or DL) without the class 4 manipulated. This means to check if the number of rows with class == 1 is equal to the number of rows with column "id" that starts with SD, DL or GL

In [12]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("csv/test/test.csv")

# Filter rows where the "id" column starts with "SD", "GL", or "DL"
filtered_df = df[df['id'].str.startswith(('SD', 'GL', 'DL'))]

# Count the number of rows in the filtered DataFrame
num_rows_with_prefix = len(filtered_df)

print("Number of rows with 'id' starting with 'SD', 'GL', or 'DL':", num_rows_with_prefix)


Number of rows with 'id' starting with 'SD', 'GL', or 'DL': 10000


In [13]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("csv/test/test.csv")

# Filter rows where the "class" column equals 1
filtered_df = df[df['class'] == 1]

# Count the number of rows in the filtered DataFrame
num_rows_with_class_1 = len(filtered_df)

print("Number of rows with 'class' equal to 1:", num_rows_with_class_1)

Number of rows with 'class' equal to 1: 10000


now filter from test_with_duplicates only the column id, fake_id and 6_way_label

In [18]:
import pandas as pd

df = pd.read_csv("C:/Users/nello/Documents/vscode_projects/Thesis/3_image_generation/generated_images/csv/test_with_duplicates.csv")

desired_columns = ['id', 'fake_id', '6_way_label']
filtered_df = df[desired_columns]

filtered_df.to_csv("csv/test/temp1.csv", index=False)

now maintain only 2 columns id and 6_way_label, where id contains id or fake_id

In [19]:
import pandas as pd

df = pd.read_csv("csv/test/temp1.csv")

# Conditionally select values for the "id" column based on the values in the "fake_id" column
df['id'] = df['fake_id'].fillna(df['id'])

# Select only the desired columns
filtered_df = df[['id', '6_way_label']]

filtered_df.to_csv("csv/test/temp2.csv", index=False)

join the temp2 csv with test.csv to collect the column 6_way_label

In [20]:
import pandas as pd

df_a = pd.read_csv("csv/test/test.csv")
df_b = pd.read_csv("csv/test/temp2.csv")

# Perform a merge operation on "id" column
merged_df = pd.merge(df_a, df_b, on="id", how="left")

merged_df.to_csv("csv/test/test.csv", index=False)